In [6]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import BaseEstimator, TransformerMixin
import joblib

In [2]:
data = pd.read_csv(r'E:\Info_Srping\FWI_Prediction\Data\processed\train_data.csv')
data

,report_date,temperature,dew_point_temperature,relative_humidity,wind_speed,wind_direction,atmospheric_pressure,visibility,precipitation,rain_days,...,duff_moisture_code,drought_code,build_up_index,initial_spread_index,fire_weather_index,daily_severity_rating,country,latitude,longitude,elevation
0,2019-08-27 12:00:00,12.0,7.6,74.4,12.0,0,1015.13,30.0,2.6,0,...,6.9,51.1,10.3,0.9,0.6,0.0,United Kingdom,54.502224,-2.68500,252.0
1,2018-06-06 12:00:00,11.1,1.7,52.0,22.1,200,1010.80,38.6,1.5,0,...,14.8,104.2,21.9,1.7,2.7,0.2,Canada,55.200000,-119.40000,745.0
2,2014-10-11 12:00:00,2.9,2.3,95.0,1.9,250,1018.19,27.7,0.0,2,...,3.1,330.1,6.1,0.1,0.0,0.0,Canada,49.820000,-74.98000,381.1
3,2015-09-11 12:00:00,22.1,11.1,49.8,9.5,267,1015.79,23.9,0.0,4,...,11.4,282.9,20.7,3.8,6.3,0.7,Canada,54.280000,-111.45000,619.0
4,2016-06-22 12:00:00,17.0,10.0,63.6,35.2,260,1016.70,16.1,0.0,1,...,107.4,482.9,138.1,15.6,47.7,25.4,United States,37.618805,-122.37542,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183935,2012-09-19 12:00:00,22.5,15.4,63.0,35.2,180,1016.90,10.1,0.0,2,...,11.5,27.6,11.5,16.9,16.3,3.8,Canada,45.230000,-63.06000,23.5
1183936,2011-08-01 12:00:00,17.4,12.1,70.0,22.2,300,1018.30,49.7,0.0,10,...,20.6,334.2,35.6,6.1,12.8,2.5,Canada,49.350000,-124.16000,14.5
1183937,2017-07-09 12:00:00,21.0,15.0,68.7,13.0,200,1014.50,14.5,0.0,1,...,5.2,107.5,9.3,2.1,1.8,0.1,Canada,43.628613,-79.39500,76.8
1183938,2010-04-24 12:00:00,16.2,-1.4,31.0,25.9,220,1016.50,48.3,0.0,3,...,12.1,21.5,12.0,15.3,15.4,3.4,Canada,45.466667,-73.75000,36.0


In [3]:
data.drop(['report_date'], axis =1 , inplace = True)

In [7]:
"""
outlier_pipeline.py  ──  RUN AS A SCRIPT OR IMPORT AS A MODULE
───────────────────────────────────────────────────────────────
Assumes you already loaded two pandas DataFrames:
    data       → training features
    test_data  → validation / test features
Edit the column lists near the bottom to match your schema.
"""
import numpy as np
import pandas as pd
from scipy.stats import boxcox
from sklearn.base import BaseEstimator, TransformerMixin
import joblib

# ───────────────────────────────────────────────
# 1.  Core transformers
# ───────────────────────────────────────────────
class BoxCoxIQR(BaseEstimator, TransformerMixin):
    """Box–Cox → IQR Boolean mask."""
    def __init__(self):
        self.lmbda_ = self.lower_ = self.upper_ = None

    def fit(self, y, *_):
        y = y.astype(float)
        if len(np.unique(y)) == 1:
            raise ValueError("BoxCoxIQR: data must not be constant.")
        if (y <= 0).any():                      # shift into (0, ∞)
            y = y + abs(y.min()) + 1e-6
        y_bc, lmbda = boxcox(y)
        q1, q3 = np.percentile(y_bc, [25, 75])
        iqr    = q3 - q1
        self.lmbda_, self.lower_, self.upper_ = lmbda, q1-1.5*iqr, q3+1.5*iqr
        return self

    def transform(self, y):
        y = y.astype(float)
        if (y <= 0).any():
            y = y + abs(y.min()) + 1e-6
        y_bc = boxcox(y, lmbda=self.lmbda_)
        return (y_bc >= self.lower_) & (y_bc <= self.upper_)

class NoTransformIQR(BaseEstimator, TransformerMixin):
    """Plain IQR mask—no transformation."""
    def fit(self, y, *_):
        q1, q3      = np.percentile(y, [25, 75])
        iqr         = q3 - q1
        self.lower_ = q1 - 1.5*iqr
        self.upper_ = q3 + 1.5*iqr
        return self
    def transform(self, y):
        return (y >= self.lower_) & (y <= self.upper_)

class ZeroInflatedIQR(BaseEstimator, TransformerMixin):
    """
    Keep zeros; BoxCox+IQR only on non-zero values.
    """
    def __init__(self):
        self.nonzero_filter = BoxCoxIQR()
    def fit(self, y, *_):
        nz = y[y != 0]
        if len(nz):
            self.nonzero_filter.fit(nz)
        return self
    def transform(self, y):
        mask        = pd.Series(False, index=y.index)
        zeros_mask  = (y == 0)
        nz          = y[~zeros_mask]
        if len(nz):
            mask.loc[nz.index] = self.nonzero_filter.transform(nz)
        return zeros_mask | mask

# ───────────────────────────────────────────────
# 2.  Column configuration  (EDIT HERE)
# ───────────────────────────────────────────────
boxcox_cols      = ['wind_speed','fine_fuel_moisture_code','drought_code',
                    'duff_moisture_code','build_up_index','fire_weather_index']
no_transform_cols = ['relative_humidity','temperature',
                     'dew_point_temperature','atmospheric_pressure']
zero_based_cols   = ['precipitation','initial_spread_index',
                     'snow_on_ground','fire_weather_index']

# Build mapping: column → transformer instance
filters = {}
for c in boxcox_cols:      filters[c] = BoxCoxIQR()
for c in no_transform_cols:filters[c] = NoTransformIQR()
for c in zero_based_cols:  filters[c] = ZeroInflatedIQR()

# ───────────────────────────────────────────────
# 3.  FIT on training data & save
# ───────────────────────────────────────────────
def fit_filters(train_df, mapping):
    mask = pd.Series(True, index=train_df.index)
    for col, f in mapping.items():
        f.fit(train_df[col])
        mask &= f.transform(train_df[col])
    return mask

train_mask = fit_filters(data, filters)
filtered_train = data.loc[train_mask].copy()
print(f"► Training rows kept: {len(filtered_train)} / {len(data)}")

joblib.dump(filters, "outlier_filters.pkl")
print("► Saved fitted filters → outlier_filters.pkl")


► Training rows kept: 1085824 / 1183940
► Saved fitted filters → outlier_filters.pkl


In [8]:
test_data = pd.read_csv(r"E:\Info_Srping\FWI_Prediction\Data\processed\test_data.csv")
test_data

,report_date,temperature,dew_point_temperature,relative_humidity,wind_speed,wind_direction,atmospheric_pressure,visibility,precipitation,rain_days,...,duff_moisture_code,drought_code,build_up_index,initial_spread_index,fire_weather_index,daily_severity_rating,country,latitude,longitude,elevation
0,2015-06-26 12:00:00,20.6,6.4,39.8,7.4,230,1016.53,24.0,0.0,1,...,9.6,20.8,9.5,4.6,4.8,0.4,Canada,44.710000,-63.630000,3.5
1,2011-08-24 12:00:00,19.0,6.0,42.0,3.7,280,1015.60,24.1,0.0,1,...,39.0,310.7,59.4,3.0,9.6,1.5,Canada,52.183056,-122.054170,939.7
2,2010-08-13 12:00:00,15.2,8.0,62.0,11.0,350,1021.90,24.1,4.3,0,...,14.3,481.2,26.6,0.4,0.5,0.0,Canada,64.110000,-117.330000,221.0
3,2015-12-13 12:00:00,3.6,-1.4,70.1,33.3,300,1004.80,19.9,7.2,0,...,0.1,0.4,0.1,0.0,0.0,0.0,Canada,47.613056,-57.610280,6.7
4,2014-11-08 12:00:00,0.5,-4.1,71.0,27.7,267,1001.24,22.9,23.3,0,...,0.1,0.0,0.0,0.0,0.0,0.0,Canada,46.780000,-65.020000,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315713,2014-07-19 12:00:00,24.0,13.0,50.0,13.0,130,1025.20,16.1,0.0,2,...,14.0,79.4,19.5,5.7,8.7,1.3,United States,43.645645,-70.308620,23.1
315714,2011-09-28 12:00:00,14.8,4.5,51.0,14.0,240,1015.74,36.2,0.0,4,...,5.0,177.0,10.0,5.1,5.0,0.5,France,43.914722,2.116389,172.0
315715,2010-08-15 12:00:00,21.0,20.0,94.0,13.0,140,1010.00,14.5,6.0,0,...,19.6,296.5,33.7,0.1,0.1,0.0,Canada,45.950000,-77.316666,130.1
315716,2019-06-30 12:00:00,26.9,16.3,52.0,9.9,32,1016.20,17.7,0.0,1,...,17.7,99.3,24.5,4.9,8.7,1.2,Canada,41.948612,-82.518890,176.8


In [9]:
# ───────────────────────────────────────────────
# 4.  LOAD & apply on test / production data
# ───────────────────────────────────────────────
def apply_filters(new_df, pkl_path="outlier_filters.pkl"):
    loaded = joblib.load(pkl_path)
    mask   = pd.Series(True, index=new_df.index)
    for col, f in loaded.items():
        mask &= f.transform(new_df[col])
    return new_df.loc[mask].copy()

filtered_test = apply_filters(test_data)
print(f"► Test rows kept: {len(filtered_test)} / {len(test_data)}")


► Test rows kept: 289834 / 315718


In [10]:
filtered_test.to_csv(r"E:\Info_Srping\FWI_Prediction\Data\processed\filtered_test_data.csv", index=False)